# 8_binary_SE_TI_Target.ipynb
Create a file with Path and SE/TI linked.
### input
- 3_Calc_Edit_Distance/output/KEGG_ID_index.csv : A file with Paths, Path IDs and Pathway map ID associated.
- 2_Paths_from_KEGG_Pathway_datafile/Paths_from_KEGG_Pathway.csv : A file with Path and Target associated. In this notebook, we only used the column 'KEGG_id' and 'HSA' in this file.
- 7_Target_Drug_SE_TI_from_SIDER_DrugBank/tbl_drug_target_TI_SE.pkl : A file with Drug and Target and TI/SE associated. In this notebook, we only used the column 'entry_id', 'SE' and 'TI' in this file.
- 7_Target_Drug_SE_TI_from_SIDER_DrugBank/Use_SE_177.csv : A SE list using for training LGBM.

### output
- 9_Integration_SE_TI_Target_datafile/Y_binary_SE.npz : A file with Path ID and SE linked.
- 9_Integration_SE_TI_Target_datafile/Y_binary_TI.npz : A file with Path ID and TI linked.
- 9_Integration_SE_TI_Target_datafile/Y_ID_name_SE.csv : A file with SE ID and SE name linked.
- 9_Integration_SE_TI_Target_datafile/Y_ID_name_TI.csv : A file with TI ID and TI name linked.

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import save_npz
from scipy.sparse import csr_matrix, coo_matrix

In [2]:
import pickle
with open('../7_Target_Drug_SE_TI_from_SIDER_DrugBank/tbl_drug_target_TI_SE.pkl', 'rb') as web:
    df_SE_TI = pickle.load(web)

In [3]:
df_SE_TI

,Drug,target_name,target_type,TI,SE,gene_name,gene_symbols,entry_id,name
0,4-AP,Potassium voltage-gated channel subfamily A me...,Protein,"['Multiple sclerosis', 'Walking disability']","['Anaphylactic shock', 'Angioedema', 'Anxiety'...",KCNA4,kcna4,hsa:3739,KCNA4
1,4-AP,Potassium voltage-gated channel subfamily C me...,Protein,"['Multiple sclerosis', 'Walking disability']","['Anaphylactic shock', 'Angioedema', 'Anxiety'...",KCNC3,kcnc3,hsa:3748,KCNC3
2,4-AP,Potassium voltage-gated channel subfamily D me...,Protein,"['Multiple sclerosis', 'Walking disability']","['Anaphylactic shock', 'Angioedema', 'Anxiety'...",KCND2,kcnd2,hsa:3751,KCND2
3,disopyramide,Potassium voltage-gated channel subfamily D me...,Protein,"['Arrhythmia', 'Bundle branch block', 'Sudden ...","['Agranulocytosis', 'Angioedema', 'Anorexia', ...",KCND2,kcnd2,hsa:3751,KCND2
4,imipramine,Potassium voltage-gated channel subfamily D me...,Protein,"['Alcoholism', 'Depression', 'Endogenous depre...","['Abdominal cramps', 'Abdominal pain', 'Adenit...",KCND2,kcnd2,hsa:3751,KCND2
...,...,...,...,...,...,...,...,...,...
2743,zinc,Tumor protein p73,Protein,"['Hepato-lenticular degeneration', 'Liver diso...","['Pancreatitis', 'Gastric irritation']",TP73,tp73,hsa:7161,TP73
2744,zonisamide,Carbonic anhydrase 13,Protein,"['Epilepsy', 'Partial seizures', 'Sudden unexp...","['Abdominal pain', 'Congenital anomaly', 'Rena...",CA13,ca13,hsa:377677,CA13
2745,zonisamide,"Carbonic anhydrase 5B, mitochondrial",Protein,"['Epilepsy', 'Partial seizures', 'Sudden unexp...","['Abdominal pain', 'Congenital anomaly', 'Rena...",CA5B,ca5b,hsa:11238,CA5B
2746,zonisamide,Carbonic anhydrase-related protein,Protein,"['Epilepsy', 'Partial seizures', 'Sudden unexp...","['Abdominal pain', 'Congenital anomaly', 'Rena...",CA8,ca8,hsa:767,CA8


In [4]:
df_SE = df_SE_TI[['SE', 'entry_id']].drop_duplicates()

In [5]:
df_SE['SE'] = df_SE['SE'].apply(eval)

In [6]:
df_use = pd.read_csv('../7_Target_Drug_SE_TI_from_SIDER_DrugBank/Use_SE_177.csv',header = 0)

In [7]:
df_use

,SE
0,Heart rate abnormal
1,Anaplastic thyroid cancer
2,Breast cancer female
3,Breast cancer invasive NOS
4,Cardiac discomfort
...,...
172,Renal impairment
173,Hepatic function abnormal
174,Renal failure acute
175,Liver function test abnormal


In [8]:
df_SE = pd.merge(df_SE.explode('SE'), df_use.reset_index(), on = 'SE').groupby(['index', 'SE'])['entry_id'].apply(list).reset_index()

In [9]:
df_SE

,index,SE,entry_id
0,0,Heart rate abnormal,[hsa:135]
1,1,Anaplastic thyroid cancer,"[hsa:1813, hsa:1814, hsa:1815]"
2,2,Breast cancer female,"[hsa:3359, hsa:1813, hsa:1139, hsa:2902]"
3,3,Breast cancer invasive NOS,"[hsa:1131, hsa:3358, hsa:1128, hsa:7442, hsa:1..."
4,4,Cardiac discomfort,"[hsa:3791, hsa:613, hsa:6714, hsa:25, hsa:4067..."
...,...,...,...
172,172,Renal impairment,"[hsa:240, hsa:1147, hsa:3551, hsa:4353, hsa:43..."
173,173,Hepatic function abnormal,"[hsa:240, hsa:4353, hsa:5468, hsa:5468, hsa:54..."
174,174,Renal failure acute,"[hsa:240, hsa:1147, hsa:3551, hsa:4353, hsa:43..."
175,175,Liver function test abnormal,"[hsa:3751, hsa:3752, hsa:3752, hsa:240, hsa:24..."


In [10]:
df_SE.to_csv('s.csv',encoding = 'utf-8')